In [6]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_6486f6925a3b4963afc058e250e1f031 = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_6486f6925a3b4963afc058e250e1f031 = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_6486f6925a3b4963afc058e250e1f031 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='w5_9Bf-0UAn0b7wzNtBiXwm1sCXUnaPtmOIG--k0OcV0',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_6486f6925a3b4963afc058e250e1f031)

body = client_6486f6925a3b4963afc058e250e1f031.get_object(Bucket='aitofindbestwelllocation-donotdelete-pr-w1t2kzuzbwx2l0',Key='dataset_with_features.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

wells = pd.read_csv(body)
wells.head()


,Unnamed: 0,longitude,latitude,target,fault0,fault1,fault2,fault3,fault4,fault5,fault6
0,0,-119.342550,38.757700,1,61.379975,10.018325,18.666062,41.437305,0.030375,42.837627,10.269944
1,1,-119.000000,39.000000,1,64.617668,5.273444,21.107012,53.641358,1.251336,69.404474,30.520319
2,2,-120.000000,39.000000,1,33.263556,5.835872,14.555254,16.701855,5.997610,13.529671,13.539749
3,3,-119.705551,39.043056,1,19.831489,3.260279,0.193372,4.876578,1.523177,16.262283,1.221805
4,4,-119.161930,38.923980,1,57.179331,3.319162,14.216486,50.574469,3.475881,54.781805,30.849068


In [7]:
wells = wells[wells.columns[1:]]
wells.head()

,longitude,latitude,target,fault0,fault1,fault2,fault3,fault4,fault5,fault6
0,-119.342550,38.757700,1,61.379975,10.018325,18.666062,41.437305,0.030375,42.837627,10.269944
1,-119.000000,39.000000,1,64.617668,5.273444,21.107012,53.641358,1.251336,69.404474,30.520319
2,-120.000000,39.000000,1,33.263556,5.835872,14.555254,16.701855,5.997610,13.529671,13.539749
3,-119.705551,39.043056,1,19.831489,3.260279,0.193372,4.876578,1.523177,16.262283,1.221805
4,-119.161930,38.923980,1,57.179331,3.319162,14.216486,50.574469,3.475881,54.781805,30.849068


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [9]:
X = wells[['fault0','fault1', 'fault2', 'fault3', 'fault4', 'fault5', 'fault6']]
y = wells['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
vanilla = LogisticRegression()
vanilla.fit(X_train, y_train)
acc_v = vanilla.score(X_test, y_test)
print(acc_v)

0.6842105263157895


Not that bad for such a small model and small dataset

We are quite sure that going with ANN with only 6 features and 186 observations is a bad idea. We will try that later when we have access to a bigger dataset.

In [11]:
from sklearn.linear_model import LogisticRegressionCV

In [12]:
choco = LogisticRegressionCV()
choco.fit(X_train, y_train)
acc_c = choco.score(X_test, y_test)
print(acc_c)

0.6842105263157895


In [13]:
from sklearn import svm

In [14]:
mclf = svm.SVC()
mclf.fit(X_train, y_train)
acc_m = mclf.score(X_test, y_test)
print(acc_m)

0.7368421052631579


In [15]:
import joblib

filename = 'finalized_model.sav'
joblib.dump(mclf, filename)

['finalized_model.sav']